MGD

In [1]:
import torch
import networkx as nx
import numpy as np
import time

# ===== problem & method settings =====
n = 2000
p = 0.5

gamma_c = 1
beta    = 0.8
alpha   = 0.0001
ITERATION_T = 150      # max inner iterations per init

exploration_parameter_eta = 2.0
covariance_matrix = exploration_parameter_eta * torch.eye(n)

# time budget per seed (seconds)
TIME_LIMIT = 20

# which seeds to run
SEEDS = range(10)      # e.g. 0–9; expand as desired

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0, 1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

# store best per seed
best_per_seed = []

for seed in SEEDS:
    # build ER graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    A     = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(G)).todense(), dtype=torch.float32)

    # compute gamma from complement degrees
    max_deg_c = max(dict(nx.complement(G).degree()).values())
    gamma = 2 + max_deg_c

    # degree-based init vector d
    degs = dict(G.degree())
    max_deg = max(degs.values())
    d = torch.tensor([1 - degs[i]/max_deg for i in range(n)], dtype=torch.float32)
    d = d / d.max()

    print(f"\nSeed {seed}: running MGD for up to {TIME_LIMIT}s")

    seed_start = time.time()
    trial = 0
    best_size = 0

    while time.time() - seed_start < TIME_LIMIT:
        # prepare initialization
        if trial == 0:
            x = d.clone()
        else:
            torch.manual_seed((seed << 16) + trial)
            x = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x.clamp_(0, 1)

        v = torch.zeros(n)  # velocity
        # run momentum gradient descent
        for it in range(ITERATION_T):
            grad = -torch.ones(n) + (gamma * A - gamma_c * A_bar) @ x
            v = beta * v + alpha * grad
            x = torch.clamp(x - v, 0, 1)

            checker, MIS = MIS_checker_efficient_3(x, A, A_bar)
            if checker:
                size = MIS.numel()
                if size > best_size:
                    best_size = size
                # print(f" trial {trial:2d} iters {it+1:4d} → MIS size {size}")
                break
        else:
            # reached ITERATION_T without finding MIS
            print(f" trial {trial:2d} iters {ITERATION_T:4d} → no MIS")

        trial += 1

    print(f"→ Seed {seed} best MIS size = {best_size}")
    best_per_seed.append(best_size)

# overall average
avg_best = np.mean(best_per_seed)
print(f"\nAverage of best MIS size across seeds: {avg_best:.2f}")


Seed 0: running MGD for up to 20s
→ Seed 0 best MIS size = 12

Seed 1: running MGD for up to 20s
 trial  1 iters  150 → no MIS
→ Seed 1 best MIS size = 12

Seed 2: running MGD for up to 20s
 trial  0 iters  150 → no MIS
 trial  1 iters  150 → no MIS
 trial  3 iters  150 → no MIS
→ Seed 2 best MIS size = 12

Seed 3: running MGD for up to 20s
 trial  0 iters  150 → no MIS
 trial  2 iters  150 → no MIS
 trial  5 iters  150 → no MIS
 trial  6 iters  150 → no MIS
→ Seed 3 best MIS size = 13

Seed 4: running MGD for up to 20s
→ Seed 4 best MIS size = 13

Seed 5: running MGD for up to 20s
 trial  1 iters  150 → no MIS
 trial  2 iters  150 → no MIS
 trial  3 iters  150 → no MIS
→ Seed 5 best MIS size = 11

Seed 6: running MGD for up to 20s
 trial  0 iters  150 → no MIS
 trial  2 iters  150 → no MIS
→ Seed 6 best MIS size = 13

Seed 7: running MGD for up to 20s
 trial  2 iters  150 → no MIS
 trial  3 iters  150 → no MIS
→ Seed 7 best MIS size = 12

Seed 8: running MGD for up to 20s
 trial  0 i

Newton

In [2]:
import torch
import networkx as nx
import numpy as np
import pandas as pd
import time

# Fixed method parameters
n = 500
p = 0.5
gamma_c = 1
exploration_parameter_eta = 2.0
max_iters = 1500
tol = 1e-8
damping0 = 1e-3
ls_iters = 10
alpha0 = 900   # fixed learning rate
beta = 0.6    # fixed backtracking scale

# Instead of N_TRIALS, run up to this many seconds per seed:
TIME_LIMIT = 5  # seconds

# Functions
def objective(x, H):
    return -x.sum() + 0.5 * x @ (H @ x)

def gradient(x, H):
    return -torch.ones_like(x) + H @ x

def MIS_checker_efficient_3(X, A, A_bar):
    Xb = X.bool().float()
    Xb_up = Xb - 0.1 * (-torch.ones_like(Xb) + (n * A) @ Xb)
    Xb_up.clamp_(0,1)
    if torch.equal(Xb, Xb_up):
        return True, torch.nonzero(Xb).squeeze()
    return False, None

def projected_newton(H, A, A_bar, x0):
    x = x0.clone()
    damping = damping0
    I = torch.eye(n, dtype=H.dtype)
    H_damped = H + damping * I
    history = []
    is_MIS = False
    MIS = None

    for it in range(max_iters):
        g = gradient(x, H)
        g_norm = g.norm().item()
        f0 = objective(x, H).item()
        history.append((it, f0, g_norm))
        if g_norm < tol:
            break

        try:
            p = torch.linalg.solve(H_damped, g)
        except RuntimeError:
            damping *= 10
            H_damped = H + damping * I
            continue

        alpha = alpha0
        accepted = False
        for _ in range(ls_iters):
            x_trial = torch.clamp(x - alpha * p, 0.0, 1.0)
            if objective(x_trial, H).item() < f0:
                x = x_trial
                accepted = True
                break
            alpha *= beta

        if not accepted:
            damping *= 10

        damping = max(damping * 0.5, 1e-7)
        H_damped = H + damping * I

        is_MIS, MIS = MIS_checker_efficient_3(x, A, A_bar)
        if is_MIS:
            break

    return x, history, is_MIS, MIS

# Main batch testing over seeds 0-9 with time‐limited trials
summaries = []
for seed in range(10):
    # Build graph for this seed
    g = nx.gnp_random_graph(n, p, seed=seed)
    A = torch.tensor(nx.adjacency_matrix(g).todense(), dtype=torch.float32)
    A_bar = torch.tensor(nx.adjacency_matrix(nx.complement(g)).todense(), dtype=torch.float32)

    # Build H
    degrees_c = dict(nx.complement(g).degree())
    gamma = 2 + max(degrees_c.values())
    H = gamma * A # - gamma_c * A_bar

    # Degree init vector
    degrees = dict(g.degree())
    max_deg = max(degrees.values())
    d = torch.tensor([1 - degrees[i]/max_deg for i in range(n)], dtype=torch.float32)
    d_init = d / d.max()

    # Covariance
    cov = exploration_parameter_eta * torch.eye(n)

    # Run as many trials as will fit in TIME_LIMIT
    trial = 0
    trial_stats = []
    print(f"ER({n}, {p}, seed {seed}): Newton with MIS-check early exit, up to {TIME_LIMIT}s")
    seed_start = time.time()
    # print(f"\nSeed {seed}: running projected_newton for up to {TIME_LIMIT}s…")
    while True:
        # stop when time budget exceeded
        if time.time() - seed_start > TIME_LIMIT:
            break

        # prepare init
        if trial == 0:
            x0 = d_init.clone()
        else:
            torch.manual_seed((seed << 16) + trial)
            x0 = torch.normal(mean=d_init, std=torch.sqrt(torch.diag(cov)))
            x0.clamp_(0.0, 1.0)

        # run Newton
        t0 = time.time()
        x_opt, history, is_MIS, MIS = projected_newton(H, A, A_bar, x0)
        elapsed = time.time() - t0

        trial_stats.append({
            'iters': len(history),
            'time': elapsed,
            'MIS_size': MIS.numel() if is_MIS else 0
        })
        # print(f" trial {trial:2d} → iters {len(history):4d}, time {elapsed:7.4f}s, MIS_size {trial_stats[-1]['MIS_size']}")
        trial += 1

    # summarize for this seed
    df = pd.DataFrame(trial_stats)
    if not df.empty:
        summaries.append({
            'seed': seed,
            'n_trials':   len(df),
            'max_MIS_size': int(df.MIS_size.max()),
            'mean_iters': df.iters.mean(),
            'mean_time':  df.time.mean()
        })
    else:
        summaries.append({
            'seed': seed,
            'n_trials':   0,
            'max_MIS_size': 0,
            'mean_iters': 0.0,
            'mean_time':  0.0
        })

# Aggregate and display results
df_summary = pd.DataFrame(summaries)
print("\nDamped Projected Newton (time‐limited) summary:")
print(df_summary)

avg_best = df_summary['max_MIS_size'].mean()
print(f"ER({n}, {p}): Newton with MIS-check early exit, up to {TIME_LIMIT}s")
print(f"\nAverage of best MIS size across seeds: {avg_best:.2f}")

ER(500, 0.5, seed 0): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 1): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 2): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 3): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 4): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 5): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 6): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 7): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 8): Newton with MIS-check early exit, up to 5s
ER(500, 0.5, seed 9): Newton with MIS-check early exit, up to 5s

Damped Projected Newton (time‐limited) summary:
   seed  n_trials  max_MIS_size  mean_iters  mean_time
0     0        29            12   80.379310   0.173502
1     1        29            11   71.793103   0.171563
2     2        31            12   73.032258   0.158385
3     3        32            11   74.281250   0.153779
4     4        22         

L-BFGS from Scipy

In [3]:
import torch
import networkx as nx
import numpy as np
from scipy.optimize import minimize
import time

# ===== your existing helper stays the same =====
class EarlyExit(Exception):
    pass

def MIS_checker_efficient_3(X_torch, adjacency_matrix_tensor, adjacency_matrix_tensor_comp):
    n_local = X_torch.shape[0]
    X_torch_binarized = X_torch.bool().float()
    X_torch_binarized_update = X_torch_binarized - 0.1 * (
        -torch.ones(n_local) + (n_local * adjacency_matrix_tensor) @ X_torch_binarized
    )
    X_torch_binarized_update = torch.clamp(X_torch_binarized_update, 0, 1)
    if torch.equal(X_torch_binarized, X_torch_binarized_update):
        MIS = torch.nonzero(X_torch_binarized).squeeze()
        return True, MIS
    return False, None

# ===== experiment settings =====
n = 500
p = 0.5
gamma_c = 1
beta = 0.8
alpha = 0.0001
ITERATION_T = 15000

# Instead of a fixed number of trials, run until this many seconds per seed:
TIME_LIMIT = 5  # seconds

SEEDS = range(10)   # <- run 50 seeds

# for across-seed summary
best_size_per_seed = []
best_trial_per_seed = []
mean_runtime_per_seed = []
mean_iters_per_seed = []

for seed in SEEDS:
    # ---- build graph for this seed ----
    G = nx.gnp_random_graph(n, p, seed=seed)
    complement_G = nx.complement(G)

    num_components = nx.number_connected_components(G)
    print(f"\n================== Seed {seed} ==================")
    print("Number of connected components:", num_components)

    A = nx.adjacency_matrix(G).todense()
    A_t = torch.tensor(A, dtype=torch.float32)
    Acomp = nx.adjacency_matrix(complement_G).todense()
    Acomp_t = torch.tensor(Acomp, dtype=torch.float32)

    # gamma depends on complement degree
    max_degree = max(dict(complement_G.degree()).values())
    gamma = 2 + max_degree

    # degree-based init d
    degrees = dict(G.degree())
    max_degree_node = max(degrees.values())
    d = torch.tensor([1 - (degrees[i] / max_degree_node) for i in range(n)], dtype=torch.float32)
    d = d / d.max()

    # H and smooth objective/grad for SciPy
    # H_np = (gamma * A_t).numpy()
    H_np = (gamma * A_t - gamma_c * Acomp_t).numpy()

    def f_np(x):
        return -np.sum(x) + 0.5 * x.dot(H_np.dot(x))

    def grad_np(x):
        return -np.ones_like(x) + H_np.dot(x)

    bounds = [(0.0, 1.0)] * n

    # per-seed collectors
    MIS_size, Iter_NO, Run_time = [], [], []

    exploration_parameter_eta = 2.0
    covariance_matrix = exploration_parameter_eta * torch.eye(n)

    print(f"ER({n}, {p}, seed={seed}): L-BFGS-B with MIS-check early exit, up to {TIME_LIMIT}s")

    seed_start_time = time.time()
    trial = 0
    while True:
        # Check time budget
        if time.time() - seed_start_time > TIME_LIMIT:
            break

        # prepare init
        if trial == 0:
            x0_np = d.numpy()
        else:
            torch.manual_seed((seed << 16) + trial)
            noise = torch.normal(mean=d, std=torch.sqrt(torch.diag(covariance_matrix)))
            x0_np = noise.numpy()

        iter_counter = {"count": 0}
        last_x = {"xk": None}

        def lbfgs_callback(xk):
            iter_counter["count"] += 1
            last_x["xk"] = xk.copy()
            x_t = torch.tensor(xk, dtype=torch.float32)
            checker, mis = MIS_checker_efficient_3(x_t, A_t, Acomp_t)
            if checker:
                raise EarlyExit()

        start_time = time.time()
        try:
            res = minimize(
                fun      = f_np,
                x0       = x0_np,
                method   = "L-BFGS-B",
                jac      = grad_np,
                bounds   = bounds,
                callback = lbfgs_callback,
                options  = {
                    "maxiter": ITERATION_T,
                    "gtol":    0.0,
                    "ftol":    0.0,
                }
            )
            total_iters = res.nit
            x_final_np = res.x
        except EarlyExit:
            total_iters = iter_counter["count"]
            x_final_np = last_x["xk"]

        x_final = torch.tensor(x_final_np, dtype=torch.float32)
        checker, mis = MIS_checker_efficient_3(x_final, A_t, Acomp_t)

        elapsed = time.time() - start_time

        if mis is not None:
            # print(f" trial {trial:2d} → MIS size {len(mis):3d}, iters {total_iters:5d}, time {elapsed:7.4f}s")
            MIS_size.append(len(mis))
            Iter_NO.append(total_iters)
            Run_time.append(elapsed)
        else:
            # print(f" trial {trial:2d} → no MIS (iters {total_iters:4d}, time {elapsed:7.4f}s)")
            pass

        trial += 1

    # per-seed summary
    if MIS_size:
        mis_sizes   = np.array(MIS_size)
        runtimes    = np.array(Run_time)
        iter_counts = np.array(Iter_NO)

        best_size = mis_sizes.max()
        best_run  = mis_sizes.argmax()
        mean_rt   = runtimes.mean()
        mean_it   = iter_counts.mean()
        p2_5_rt, p97_5_rt = np.percentile(runtimes, [2.5, 97.5])
        p2_5_it, p97_5_it = np.percentile(iter_counts, [2.5, 97.5])
        
        '''print(f"— Seed {seed} summary —")
        print(f"Best MIS size: {best_size} (trial {best_run})")
        print(f"Runtime (s): mean={mean_rt:.4f}, min={runtimes.min():.4f}, max={runtimes.max():.4f}, "
              f"std={runtimes.std(ddof=1):.4f}, 95% CI=({p2_5_rt:.4f}, {p97_5_rt:.4f})")
        print(f"Iters: mean={mean_it:.1f}, min={iter_counts.min()}, max={iter_counts.max()}, "
              f"std={iter_counts.std(ddof=1):.1f}, 95% CI=({p2_5_it:.1f}, {p97_5_it:.1f})")'''

        best_size_per_seed.append(best_size)
        best_trial_per_seed.append(int(best_run))
        mean_runtime_per_seed.append(float(mean_rt))
        mean_iters_per_seed.append(float(mean_it))
    else:
        print(f"— Seed {seed} summary — no MIS flagged by checker.")
        '''best_size_per_seed.append(None)
        best_trial_per_seed.append(None)
        mean_runtime_per_seed.append(None)
        mean_iters_per_seed.append(None)'''

# across-seeds summary
print(f"ER({n}, {p}): L-BFGS-B with MIS-check early exit, up to {TIME_LIMIT}s")
vals = [v for v in best_size_per_seed if v is not None]
if vals:
    vals = np.array(vals)
    print("\n=========== Across-seeds summary (best per seed) ===========")
    print(f"Seeds with MIS: {len(vals)}/{len(SEEDS)}")
    # print(f"Best MIS size: max={vals.max()}, min={vals.min()}, mean={vals.mean():.2f}, std={vals.std(ddof=1):.2f}")
    print(f"Mean MIS size: {vals.mean():.2f}")


================== Seed 0 ==================
Number of connected components: 1
ER(500, 0.5, seed=0): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 1 ==================
Number of connected components: 1
ER(500, 0.5, seed=1): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 2 ==================
Number of connected components: 1
ER(500, 0.5, seed=2): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 3 ==================
Number of connected components: 1
ER(500, 0.5, seed=3): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 4 ==================
Number of connected components: 1
ER(500, 0.5, seed=4): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 5 ==================
Number of connected components: 1
ER(500, 0.5, seed=5): L-BFGS-B with MIS-check early exit, up to 5s

================== Seed 6 ==================
Number of connected components: 1
ER(500, 0.5, seed=6): L-BFGS-B with MI